# Analyse en Composantes Principales (les calculs)

## Données : les iris de fisher

https://fr.wikipedia.org/wiki/Iris_de_Fisher

### Le nuage de points : données brutes

Les données que l'on considère seront $n$ individus décrits par $p$ variables quantitatives. On peut ainsi considérer que nos données sont des points d'un espace gométrique à $p$ dimensions. On a contume d'appeler l'ensembre de ces points le *nuage*. 

On *range* alors nos données dans une matrice $X$ à $n$ lignes et $p$ colonnes où :
- chaque donnée $x_i = (x_i^1, \dots, x_i^p)$ est la ligne $i$ de $X$,
- chaque attribut $x^j=(x_1^j, \dots, x_n^j)$ $x^j$ est la colonne $j$ de $X$.

Cette interprétation géométrique va se révéler féconde tant au niveau de l'interprétation des données que des différents calculs.

In [ ]:
import seaborn as sns

In [ ]:
X = sns.load_dataset('iris').drop(columns="species")
X

### Centrage et réduction du nuage

Pour permettre une interprétation géométrique des données, la première chose à faire est **TOUJOURS** de les centrer et réduire.

Notez que le centre de gravité (*ie.*
la moyenne) et les variances se calculent sur les attributs (les $x^j$) :
- moyenne d'un axe : $\overline{x^j} = \frac{1}{n} * \sum_{i}x_i^j$,
- variance d'un axe : $\sigma(x^j)^2 = (1/n) * \sum_i ({x}^j_i - \overline{{x}^j})^2$.

#### Centrer les données

On applique la transformation suivante à nos données : 
$${x'}^j_i = x_i^j - \overline{x^j}$$
La moyenne des individus $x'_i$ sur chaque axe est alors 0.

L'origine de l'espace est ainsi la moyenne g = $(\overline{x'^1}, \dots, \overline{x'^p}) = (0, \dots, 0)$ du nuage de points. 


Chaque axe contient à la fois des données positives (> que la moyenne sur cet axe) et négative (< à la moyenne sur cet axe). Les axe et leurs 0 montrent une tendance des données.

#### Réduire les données centrées

On applique la transformation suivante à nos données : 
$${x''}^j_i = \frac{{x'}_i^j}{ \sigma({x^j})}$$
La moyenne des ${x''}^j$ est toujours 0 et sa variance est alors 1 :
$$\sigma({x''}^j)^2 = (1/n) * \sum_i ({x''}^j_i - \overline{{x''}^j})^2 = (1/n) * \sum_i (\frac{{x'}^j_i}{ \sigma({x^j})})^2 = (1/\sigma({x^j})^2) * ((1/n) * \sum_i ({x}^j_i - \overline{{x}^j})^2) = (1/\sigma({x^j})^2) * \sigma({x^j})^2 = 1$$

#### Centrer et réduire les iris

On peut faire les 2 opérations en une seule fois en transformant nos données telles que : 
    $${x}_i^j \leftarrow \frac{(x_i^j - \bar{x^j})} {\sigma(x^j)}$$

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas

In [ ]:
scaler = StandardScaler()

X = pandas.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
X.describe()

In [ ]:
X

#### Interprétation géométrique

La dispersion des points selon chaque axe est maintenant de la même amplitude (c'est la variance) autour de l'origine : chaque axe participe donc de la même manière à la position de l'individu dans le nuage. Il n'y a plus d'axe prépondérant par rapport à un autre.

En effet, l'importance de chaque individu $x_i$ est sa *longueur* $||x_i||^2$ : plus il est éloigné de l'origine plus il se distingue de la moyenne et donc plus il apporte d'information.
Comme nos données sont centrées et réduites, on a : 

$$||x_i||^2  = \sum_j ({x''}^j_i)^2 = \sum_j 1 = p$$

Chaque individu à la même longueur (qui correspond à son importantce dans le nuage) et chaque axe contribue de la même manière à cette longueur (il contribue de 1). Notre nuage est maintenant composé d'individu d'égale importance et il n'y a pas d'axe prépondérant. 

La **quantité d'information** totale du nuage est alors la somme des longueurs des individus. C'est **l'inertie** :
 
$$I = (1/n) * \sum_{i=1}^n ||x_i||^2 = p$$

#### Corrélation entre les axes

Si nos données sont centrées et réduite, la correlation linéaire entre deux axes est égale à sa covariance et peut s'écrire :

$$\mbox{cov}(x^i, x^j) = \mbox{cor}(x^i, x^j) = {^tx^i}Dx^j$$

où $D$ est une matrice diagonale contenant les poids de chaque individu ($1/n$ pour chaque individu par défaut).

La corrélation linéaire entre 2 axes pour notre population nous permet plusieurs analyses :
- on peut comparer les corrélations entre elle car : $ -1\leq \mbox{cor}(x^i, x^j) \leq 1$ pour tous $i$ et $j$.
- si la corrélation est proche de -1 l'axe $i$ et $j$ fonctionnent de **façon opposées** pour notre population : lorsque l'un monte l'autre descent et réciproquement
- si la corrélation est proche de +1 l'axe $i$ et $j$ fonctionnent de la **même façon** pour notre population : lorsque l'un monte l'autre monte également et réciproquement
- si la corrélation est proche de 0 l'axe $i$ et $j$ sont **indépendants** pour notre population : lorsque l'un monte l'autre monte ou descent et réciproquement

In [ ]:
X.corr()

En notant $D = \frac{1}{n} * I_n$ ($I_n$ est la matrice identité à $n$ lignes) on peut trouver les variances et covariances des différents attributs matriciellement:
$$R= {^tX}DX = \left(
\begin{array}{cccccc}
1&\dots &\mbox{cor}(x^1,x^j)&\dots &\mbox{cor}(x^1,x^p)\\
     &\ddots&      &      &\vdots\\
     &      &1 &\mbox{cor}(x^i,x^j)&\mbox{cor}(x^i,x^p)\\
     &      &      &\ddots&\vdots\\
     &      &      &      &1
\end{array}
\right)$$


## Ce que l'on veut faire

On essaie de trouver une droite $\mathcal{D}$ minimisant $\sum_{i=1}^n \frac{1}{n}||x_i - p_{\mathcal{D}}(x_i)||^2$
avec la norme euclidienne: $||x_i||^2 = x_i {^t x_i}$

Après quelques calculs fait au tableau et en notant $g$ le centre de gravité, on trouve :
$$\sum_{i=1}^n \frac{1}{n}||x_i - p_{\mathcal{D}}(x_i)||^2 = \sum_{i=1}^n \frac{1}{n}||x_i||^2 + ||p_{\mathcal{D}}(g)||^2- \sum_{i=1}^n \frac{1}{n}||p_{\mathcal{D}}(x_i) - p_{\mathcal{D}}(g)||^2$$

Nos données étant centrées, $g$ est égale à l'origine, il est donc équivalent de :
- minimiser $\sum_{i=1}^n \frac{1}{n}||x_i - p_{\mathcal{D}}(x_i)||^2$
- maximiser $\sum_{i=1}^n \frac{1}{n}||p_{\mathcal{D}}(x_i)||^2$

## Droite maximisant les projections

### La trouver

Si $u$ est un vecteur directeur (en colonne) de la droite $\mathcal{D}$ on a que $||p_{\mathcal{D}}(x_i)|| = |x_i u|$. De là:
$$\sum_{i=1}^n \frac{1}{n}||p_{\mathcal{D}}(x_i)||^2 = ^t(Xu)D(Xu) = ^tu (^tXDX)u = {^tu} R u$$


On a donc :
- Si $u$ est un vecteur propre de $R$ on a $Ru = \lambda u$ et de là ${^tu} R u = \lambda {^t u}u = \lambda ||u||^2$. On en déduit que $\lambda \geq 0$ pour tout vecteur propre.
- $R$ est symétrique

$R$ est ainsi une matrice semi-définie positive : ses vecteurs propres forment une base orthogonoale de $\mathbb{R}^p$. Soient $u_1, \dots u_p$ les vecteurs propres normés de $R$ associées aux valeurs propres $\lambda_1 \geq \dots \geq \lambda_p$. Tout vecteur unitaire $u$ peut donc s'écrire $u = \sum_{i=1}^p \alpha_i u_i$ et de là :

$$ {^tu} R u  = {^t(\sum_{i=1}^p} \alpha_i u_i) R (\sum_{i=1}^p \alpha_i u_i) = \sum_{i, j} \lambda_j \alpha_i \alpha_j {^tu_i} u_j = \sum_{i} \lambda_i \alpha_i^2 \leq \lambda_1 \sum_i \alpha_i^2 = \lambda_1$$

La droite maximisant les projections est celle de vecteur directeur le vecteur propre de $R$ associé à la plus grande de ses valeurs propres.

### Qu'en faire ?

#### Qualité de la projection !

L'information totale du nuage est notre inertie $I = (1/n) * \sum_{i=1}^n ||x_i||^2$ pour quantifier la qualité de l'approximation, c'est à dire la part d'information véhiculée par l'axe on calcule l'inertie des points projetés sur $\mathcal{D}$ : 

$$I_{\mathcal{D}} =\sum_{i=1}^n \frac{1}{n}||p_{\mathcal{D}}(x_i)||^2$$

On a bien-sûr $I \geq I_{\mathcal{D}}$ quelle que soit la droite considérée, et l'inertion conservée maximum peut se calculer :

$$ \sum_{i=1}^n\frac{1}{n}||p(x_i)||^2 = {^tXu_1}DXu_1 = {^tu_1} R u_1 = \lambda_1$$

Comme l'inertie totale $I$ vaut $p$, le pourcentage d'information véhiculé par l'axe est $\lambda_1 /p$.

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
from sklearn.decomposition import PCA 

pca = PCA(n_components=1) #un unique vecteur (droite maximisant les projections)
pca.fit(X)

In [ ]:
print(pca.explained_variance_ratio_) #information véhiculée
print(pca.components_) #vecteur propre

73% de l'information est expliquée par cette droite. Les coordonnées du vecteur directeur selon les anciens axes nous montre les contributions de chaque axe à ce vecteur. Chaque ancien axe explique 25% de l'inertie (les données sont réduites), ce qui est bien moins que notre nouvel axe.

In [ ]:
inertie_totale = len(X.columns)

print(inertie_totale)
for column in X:
    print((sum(x**2 for x in X[column]) / len(X[column])) / inertie_totale)

#### Dessiner les projections

Les coordonnées des iris sur l'axe de la droite sont égaux à $c_1 = Xu_1$

In [ ]:
import numpy

U = numpy.transpose(pca.components_)
C = pandas.DataFrame(X.dot(U))

In [ ]:
C

In [ ]:
# dessins matplotlib + stype seaborn

import matplotlib.pyplot as plt

sns.set()

In [ ]:
# palette de couleurs

current_palette = sns.color_palette()
sns.palplot(current_palette)

In [ ]:
# une couleur pour chaque classe d'iris

colors = [current_palette[3], current_palette[8], current_palette[2]]
iris_colors = [colors[i // 50] for i in range(len(X))]

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

C.plot.scatter(0, 0, color=iris_colors, ax=ax)

plt.show() 

#### Lien avec les anciens axes

Pour analyser plus finement comment cet axe a été construit on regarde les corrélations linéaires entre les coordonnées des points selon les anciens axes (les $x^j$) et les coordonnées selon le nouvel axe (les projections) : 
$\mbox{cor}(c_1, x^j)$

In [ ]:
for column in X:
    print(column, C[0].corr(X[column]))

## Trouver un changement de repère maximisant les inerties.

On peut montrer que si on cherche un sous espace à $k$ dimensions maximisant les projections, ce sous espace est composé des $k$ premiers verteurs propres de $R$. On peut aisni faire un changement de coordonnée de nos données dans la base des vecteurs propres de $R$, ces axes étant plus proche des données que les axes initiaux.

Ces axes sont appelés "axes principaux".

In [ ]:
pca = PCA()
pca.fit(X)

U = numpy.transpose(pca.components_) # vecteurs propres
C = pandas.DataFrame(X.dot(U)) # nouvelles coordonnées

I = pandas.DataFrame(pca.explained_variance_ratio_)  # information véhiculée

### Inertie expliquée

L'information véhiculée par les axes principaux est donc de plus en plus petite.

In [ ]:
I # pourcentage inertie de chaque axe

Les 2 premiers axes expliquant à eux seuls 96% de l'information.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.plot(ax=ax)

plt.title("inertie expliquee")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquee")

plt.show()

L'inertie cumulée nous permet de voir quels sont les axes que l'on peut garder sans perdre trop d'information (réduction de dimension).

In [ ]:
I.cumsum() # inertie cumulée

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.cumsum().plot(ax=ax)

plt.title("inertie expliquee cumulée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquee")

plt.show()

### Projections

In [ ]:
C

Deux axes suffisent ici pour représenter presque parfaitmeent nos données (ie. très peu de perte d'information)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

C.plot.scatter(0, 1, color=iris_colors, ax=ax)

plt.show() 

## Contributions aux axes

### Correlations des nouveaux axes aux anciens

In [ ]:
correlations = pandas.DataFrame([[C[facteur].corr(X[column]) for facteur in C] for column in X], index=X.columns)

In [ ]:
correlations

L'interprétation se fait en ne gardant que ceux qui contribuent le plus aux axes.
- Pour l'axe 1 : + petal width, + petal length, + sepal length
- Pour l'axe 2 : - sepal width

En "gros" : l'axe 1 sépare le rouge des autres et l'axe 2 le vert du jaune.


### Cercle des corrélations

Comme toutes les corrélations sont comprisent entre -1 et 1, si l'on représente les corrélations des anciens axes à 2 nouveaux axes, ils sont tous dans le cercle unité.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

draw = correlations.plot.scatter(x=0, y=1, ax=ax, color="red")


for index, row in correlations.iterrows():
    draw.annotate(str(index), (row[0], row[1]))

draw.add_patch(plt.Circle((0, 0), radius=1, color=(0, 0, 0, .3)))

plt.xlim(-1.2, 1.2)
plt.ylim(-1.2, 1.2)
plt.axvline(0)
plt.axhline(0)

plt.show()

## Qualité de la projection

Ce n'est pas parce que globalement les axes sont les meilleurs qu'individuellement certains points ne se projettent pas bien sur les axes. Il ne faut pas analyser un point si ses projections sur les axes considérés sont mauvaises.

On repère ces points s'ils sont proches des axes (projection presque nulle). On peut également regarder le rapport entre la norme du projeté sur la norme du point dont le carré correspond au cosinus carré l'angle $\theta$ entre les 2 vecteur : $cos(\theta)^2 = \frac{||p(x_i)||^2}{||x_i||^2}$


In [ ]:
normes = (X**2).sum(axis=1)
quality = (C**2).div(normes, axis='index')

In [ ]:
quality

In [ ]:
quality.sum(axis=1) # qualité projection sur les 4 axes

In [ ]:
quality[[1, 3]].sum(axis=1) # qualité projection sur les axes 1 et 3

angle en degré pour les 2 premiers axes : 

In [ ]:
# angle: acos(sqrt(p))

from math import sqrt, acos, pi

quality[[0, 1]].sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi)

In [ ]:
quality[[0, 1]].sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi).sort_values()